<a href="https://colab.research.google.com/github/ucaokylong/Tabular_data/blob/main/TimeSeries_Classification_Conv1D_interpolateNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# interpolate

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!gdown --id 1SBR55ZwlblX7diGvBzPlI-9BAY-92WyX

!gdown --id 1Ow3cjBVHxiRZfGDIa1hQtkRtbFJIt9X_

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('train_nan.csv', header=None)
df.head()

In [ ]:
df = df.interpolate(axis=1)
df.head()

In [ ]:
df = df.to_numpy()
y_train = df[:, 0]
x_train = df[:, 1:]
y_train = y_train.astype(int)

In [ ]:
df = pd.read_csv('/content/test_nan.csv', header=None)
df.head()

In [ ]:
df = df.interpolate(axis=1)
df.head()

In [ ]:
df = df.to_numpy()
y_test = df[:, 0]
x_test = df[:, 1:]
y_test = y_test.astype(int)

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))

plt.figure()
for c in classes:
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
num_classes = len(np.unique(y_train))
print(num_classes)

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

## Build a model

In [ ]:
input_layer = keras.layers.Input((500, 1))

conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.ReLU()(conv1)

conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.ReLU()(conv2)

conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.ReLU()(conv3)

gap = keras.layers.GlobalAveragePooling1D()(conv3)
output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

## Train the model

In [ ]:
epochs = 200
batch_size = 32

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test, y_test),
    verbose=1,
)

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()